In [1]:
import pandas as pd
import os
from PIL import Image

In [ ]:
excel_path = "D:\Project Sherlock\Grafiki\Grafiki total\Generated_maps.xlsx"
images_dir = "D:\Project Sherlock\Grafiki\Grafiki total\\images"
labels_dir = "D:\Project Sherlock\Grafiki\Grafiki total\\labels"

In [5]:
os.listdir("D:\Project Sherlock\Grafiki\Grafiki total\dataset")

['No_pc_0.jpg',
 'No_pc_1.jpg',
 'No_pc_10.jpg',
 'No_pc_100.jpg',
 'No_pc_101.jpg',
 'No_pc_102.jpg',
 'No_pc_103.jpg',
 'No_pc_104.jpg',
 'No_pc_105.jpg',
 'No_pc_106.jpg',
 'No_pc_107.jpg',
 'No_pc_108.jpg',
 'No_pc_109.jpg',
 'No_pc_11.jpg',
 'No_pc_110.jpg',
 'No_pc_111.jpg',
 'No_pc_112.jpg',
 'No_pc_113.jpg',
 'No_pc_114.jpg',
 'No_pc_115.jpg',
 'No_pc_116.jpg',
 'No_pc_117.jpg',
 'No_pc_118.jpg',
 'No_pc_119.jpg',
 'No_pc_12.jpg',
 'No_pc_120.jpg',
 'No_pc_121.jpg',
 'No_pc_122.jpg',
 'No_pc_123.jpg',
 'No_pc_124.jpg',
 'No_pc_125.jpg',
 'No_pc_126.jpg',
 'No_pc_127.jpg',
 'No_pc_128.jpg',
 'No_pc_129.jpg',
 'No_pc_13.jpg',
 'No_pc_130.jpg',
 'No_pc_131.jpg',
 'No_pc_132.jpg',
 'No_pc_133.jpg',
 'No_pc_134.jpg',
 'No_pc_135.jpg',
 'No_pc_136.jpg',
 'No_pc_137.jpg',
 'No_pc_138.jpg',
 'No_pc_139.jpg',
 'No_pc_14.jpg',
 'No_pc_140.jpg',
 'No_pc_141.jpg',
 'No_pc_142.jpg',
 'No_pc_143.jpg',
 'No_pc_144.jpg',
 'No_pc_145.jpg',
 'No_pc_146.jpg',
 'No_pc_147.jpg',
 'No_pc_148.jpg',
 

In [ ]:

os.makedirs(labels_dir, exist_ok=True)

# Wczytaj dane
df = pd.read_excel(excel_path)

# Grupuj po nazwie pliku
grouped = df.groupby("filename")

for filename, group in grouped:
    # Otwórz obraz, żeby pobrać rozmiary
    img_path = os.path.join(images_dir, filename)
    if not os.path.exists(img_path):
        print(f"Brak obrazu: {img_path}, pomijam...")
        continue

    img = Image.open(img_path)
    img_w, img_h = img.size

    yolo_lines = []

    for _, row in group.iterrows():
        x = row['x']
        y = row['y']
        w = row['width']
        h = row['height']

        x_center = x + w / 2
        y_center = y + h / 2

        x_center_norm = x_center / img_w
        y_center_norm = y_center / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        yolo_line = f"0 {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
        yolo_lines.append(yolo_line)

    # Zapisz plik txt
    txt_filename = os.path.splitext(filename)[0] + ".txt"
    txt_path = os.path.join(labels_dir, txt_filename)

    with open(txt_path, "w") as f:
        f.write("\n".join(yolo_lines) + "\n")

    print(f"Utworzono: {txt_path}")


In [20]:
# Liczba plików
num_files = 750
counter = 0
output_dir = ("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset")
for i in range(0,num_files):
    filename = "No_pc_{}.txt".format(str(counter))
    file_path = os.path.join(output_dir, filename)
    
    # Tworzymy pusty plik
    with open(file_path, "w") as f:
        pass
    counter += 1
   # print(i)

print(f"✅ Utworzono {num_files} pustych plików w folderze {output_dir}")


✅ Utworzono 750 pustych plików w folderze D:\Project Sherlock\Grafiki\Grafiki total\dataset


In [21]:
# === Funkcja do zapisu labeli YOLO ===
def create_yolo_labels(filename, group, split_labels_dir, img_path):
    img = Image.open(img_path)
    img_w, img_h = img.size

    yolo_lines = []
    for _, row in group.iterrows():
        x = row['x']
        y = row['y']
        w = row['width']
        h = row['height']

        x_center = x + w / 2
        y_center = y + h / 2

        x_center_norm = x_center / img_w
        y_center_norm = y_center / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        yolo_line = f"0 {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
        yolo_lines.append(yolo_line)

    # Zapis
    txt_filename = os.path.splitext(filename)[0] + ".txt"
    txt_path = os.path.join(split_labels_dir, txt_filename)

    with open(txt_path, "w") as f:
        f.write("\n".join(yolo_lines) + "\n")

In [ ]:
import os
import shutil
import random
import pandas as pd
from PIL import Image

# === ŚCIEŻKI ===
excel_path = "maps_points.xlsx"
images_with_players_dir = "images_with_players"
empty_images_dir = "images_empty"
dataset_dir = "dataset"
images_dir = os.path.join(dataset_dir, "images")
labels_dir = os.path.join(dataset_dir, "labels")

# === PROPORCJA PODZIAŁU ===
train_ratio = 0.8

# === TWORZENIE FOLDERÓW ===
for split in ["train", "val"]:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, split), exist_ok=True)

# === Wczytanie danych z Excela ===
df = pd.read_excel(excel_path)

# === Listy plików ===
player_files = df["filename"].unique().tolist()
empty_files = [f for f in os.listdir(empty_images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# === Shuffle i podział ===
random.shuffle(player_files)
random.shuffle(empty_files)

split_idx_p = int(len(player_files) * train_ratio)
split_idx_e = int(len(empty_files) * train_ratio)

train_player_files = player_files[:split_idx_p]
val_player_files = player_files[split_idx_p:]

train_empty_files = empty_files[:split_idx_e]
val_empty_files = empty_files[split_idx_e:]



# === PRZETWARZANIE PLIKÓW Z GRACZAMI ===
for split, files in [("train", train_player_files), ("val", val_player_files)]:
    split_images_dir = os.path.join(images_dir, split)
    split_labels_dir = os.path.join(labels_dir, split)

    grouped = df[df["filename"].isin(files)].groupby("filename")

    for filename, group in grouped:
        src_img_path = os.path.join(images_with_players_dir, filename)
        dst_img_path = os.path.join(split_images_dir, filename)

        if not os.path.exists(src_img_path):
            print(f"Brak obrazu: {src_img_path}, pomijam...")
            continue

        # Kopiuj obraz
        shutil.copy(src_img_path, dst_img_path)

        # Stwórz label
        create_yolo_labels(filename, group, split_labels_dir, src_img_path)

# === PRZETWARZANIE PUSTYCH OBRAZÓW ===
for split, files in [("train", train_empty_files), ("val", val_empty_files)]:
    split_images_dir = os.path.join(images_dir, split)
    split_labels_dir = os.path.join(labels_dir, split)

    for filename in files:
        src_img_path = os.path.join(empty_images_dir, filename)
        dst_img_path = os.path.join(split_images_dir, filename)

        shutil.copy(src_img_path, dst_img_path)

        # Pusty plik txt
        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(split_labels_dir, txt_filename)
        with open(txt_path, "w") as f:
            pass

# === TWORZENIE data.yaml ===
yaml_content = f"""
train: {os.path.abspath(os.path.join(images_dir, "train"))}
val: {os.path.abspath(os.path.join(images_dir, "val"))}

nc: 1
names: ['crewmate']
"""

with open(os.path.join(dataset_dir, "data.yaml"), "w") as f:
    f.write(yaml_content.strip())

print("✅ Zrobione! Dataset gotowy do YOLOv8 🎮🚀")
